## Task 1 - Create a dataset  for further analysis

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from typing import List

### Preprocessing customer information dataset

In [2]:
# Load customer information dataset
customer_info = pd.read_csv("customer_info.csv")

# Print information about data types and missing values
customer_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68529 entries, 0 to 68528
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RANDOM_ID        68529 non-null  int64  
 1   ONBOARDING_YEAR  50424 non-null  float64
 2   AGE              68529 non-null  int64  
 3   LCR_PERCENT      68529 non-null  int64  
 4   GENDER_CD        56068 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 2.6 MB


To fill missing information regarding customer's gender I am going to simply create an additional category - 0. For ONBOARDING_YEAR I will fill missing values with a **mean of each age group** as these variables are highly correleted.

In [3]:
# Fill missing gender information with 0 (new category)
customer_info.GENDER_CD.fillna(0, inplace=True)

# Fill missing values with mean of each age group
customer_info.ONBOARDING_YEAR = customer_info.ONBOARDING_YEAR.fillna(customer_info.groupby('AGE')['ONBOARDING_YEAR'].transform('mean'))
customer_info.ONBOARDING_YEAR = customer_info.ONBOARDING_YEAR.astype('int')

In [4]:
# Create features based on information values of RANDOM_ID
def get_first_digit(number):
    while (number >= 10):
        number = number // 10
    return number

def get_last_three_digits(number):
    return str(number)[-4:]


customer_info["GROUP_1"] = customer_info.RANDOM_ID.apply(get_first_digit)
customer_info["GROUP_2"] = customer_info.RANDOM_ID.apply(get_last_three_digits)

### Preprocessing channels usage dataset

In [5]:
channels_use = pd.read_csv("Channels_use.csv")
channels_use.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310994 entries, 0 to 310993
Data columns (total 27 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   RANDOM_ID        310994 non-null  int64 
 1   DATE_FROM        310994 non-null  object
 2   DATE_TO          310994 non-null  object
 3   YEAR             310994 non-null  int64 
 4   MONTH            310994 non-null  int64 
 5   ATM_VISITS       310994 non-null  int64 
 6   ATM_FJ_UTTEKT    310994 non-null  int64 
 7   ATM_AM_UTTEKT    310994 non-null  int64 
 8   ATM_FJ_INNLOGN   310994 non-null  int64 
 9   ATM_AM_INNLOGN   310994 non-null  int64 
 10  APP_VISITS       310994 non-null  int64 
 11  APP_FJ_MILLIF    310994 non-null  int64 
 12  APP_AM_MILLIF    310994 non-null  int64 
 13  NETB_VISITS      310994 non-null  int64 
 14  NETB_FJ_MILLIF   310994 non-null  int64 
 15  NETB_AM_MILLIF   310994 non-null  int64 
 16  UTIBU_VISITS     310994 non-null  int64 
 17  UTIBU_FJ_M

In [6]:
# Set up a correct types for dates
channels_use["DATE_FROM"] = pd.to_datetime(channels_use["DATE_FROM"], dayfirst=True)
channels_use["DATE_TO"] = pd.to_datetime(channels_use["DATE_TO"], dayfirst=True)

# Rename date columns
channels_use.rename(columns={"DATE_FROM":"CHANNEL_DATE_FROM", "DATE_TO":"CHANNEL_DATE_TO"}, inplace=True)

I am going to summaraize this dataframe based on the RANDOM_ID. Columns representing channel usage will be represent as a sum or count. Features that are related to activity will be presented as a mean of the preriod. Columns that contain date information will be represented accordingly to the character of the information.  

In [7]:
# Divide columns into summary category (sum/mean)
sum_columns = []
mean_columns = []

sum_columns.append("ATM_VISITS")

for column in channels_use.columns:
    if "_FJ_" in column or "_AM_" in column:
        sum_columns.append(column)
    elif "ACTIVE" in column:
        mean_columns.append(column)

# Create summary datasets
acitivity = channels_use.groupby("RANDOM_ID")[mean_columns].mean().reset_index()
channels = channels_use.groupby("RANDOM_ID")[sum_columns].sum().reset_index()
dates = channels_use.groupby("RANDOM_ID")[["CHANNEL_DATE_FROM", "CHANNEL_DATE_TO"]].agg({"CHANNEL_DATE_FROM":min,
                                                                                         "CHANNEL_DATE_TO":max}).reset_index()

# Merge summary datasets
channel_summary = acitivity.merge(right=channels, how="outer", on="RANDOM_ID").merge(right=dates, how="outer", on="RANDOM_ID")

# Merge channel summary with customer information
channel_summary = customer_info.merge(right=channel_summary, how="outer", on="RANDOM_ID")

In [8]:
# Fill missing values with 0 and dummy date
channel_summary[["CHANNEL_DATE_FROM", "CHANNEL_DATE_TO"]] = channel_summary[["CHANNEL_DATE_FROM", "CHANNEL_DATE_TO"]].fillna(value=pd.to_datetime('1/1/2015'))
channel_summary.fillna(0, inplace=True)

# Convert float to int to summariaze activity as binary indicator
for column in channel_summary.columns:
    if channel_summary[column].dtype == "float":
        channel_summary[column] = channel_summary[column].astype('int')

### Product usage preprocessing

In [9]:
product_use = pd.read_csv("Product_use.csv")
product_use.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2388329 entries, 0 to 2388328
Data columns (total 6 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   DATE_FROM       object
 1   DATE_TO         object
 2   RANDOM_ID       int64 
 3   PRODUCT_KEY_ID  int64 
 4   VARIABLE_ID     int64 
 5   VAR_NUMBER      int64 
dtypes: int64(4), object(2)
memory usage: 109.3+ MB


In [10]:
# Change type of date information to datatime
product_use["DATE_FROM"] = pd.to_datetime(product_use["DATE_FROM"], dayfirst=True)
product_use["DATE_TO"] = pd.to_datetime(product_use["DATE_TO"], dayfirst=True)

product_use.rename(columns={"DATE_FROM":"PRODUCT_DATE_FROM", "DATE_TO":"PRODUCT_DATE_TO"}, inplace=True)

### Improting variable and product information

In [11]:
variable_info = pd.read_csv("Variable_info.csv", encoding='latin-1', sep=";")
variable_info

,ID,BREYTA,UTSKYRING,TEGUND
0,1,Innlansreikningur,Hvort viðkomandi sé með innlánsreikning af til...,Binary
1,5,Velta_innlansreikning,upphæð innlagnar inn á innlánsreikning á gefnu...,Amount
2,34,Kreditkort,Var viðkomandi með notað kreditkort á gefnu tí...,Binary
3,36,velta_1m,Upphæð veltu korts á gefnu tímabili,Amount
4,50,Er_med_utlan,Er með útlán af tilteknu vörunúmer á gefnu tím...,Binary
5,53,Skuldabréfastaða,Útlánastaða viðkomandi viðskiptavinar (innan e...,Amount
6,424,Asset under mgmt,Er með eignir í stýringu á gefnu tímabili,Binary
7,446,Asset under mgmt,Staða eigna í stýringu á gefnu tímabili,Amount


In [12]:
product_info = pd.read_csv("product_info.csv", encoding='latin-1', sep=",")
product_info.head()

,PRODUCT_KEY_ID,PRODUCT_GROUP_PARENT,PRODUCT_GROUP
0,1070,Innlán,Sparnaðarreikningur
1,1176,Innlán,Sparnaðarreikningur
2,270,Innlán,Sparnaðarreikningur
3,1141,Innlán,Tékkareikningur
4,1126,Innlán,Tékkareikningur


### Merging datasets into one file

In [13]:
# Merge product usage dataset with variable information
dataset = product_use.merge(right=variable_info, how="left", left_on='VARIABLE_ID', right_on='ID')

# Merge product usage dataset with product information
dataset = dataset.merge(right=product_info, how="left", on="PRODUCT_KEY_ID")

# Merge product usage dataset with customer and channel usage information
dataset = dataset.merge(right=channel_summary, how="left", on="RANDOM_ID")

### Optimizing memory usage 

In [14]:
# Source: https://medium.com/bigdatarepublic/advanced-pandas-optimize-speed-and-memory-a654b53be6c2
def optimize_numbers(df):
    ints = df.select_dtypes(include=['int64']).columns.tolist()
    df[ints] = df[ints].apply(pd.to_numeric, downcast='integer')
    return df

def optimize_strings(df, datetime_features):
    for col in df.select_dtypes(include=['object']):
        if col not in datetime_features:
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if float(num_unique_values) / num_total_values < 0.5:
                df[col] = df[col].astype('category')
        else:
            df[col] = pd.to_datetime(df[col])
    return df

In [15]:
dataset = optimize_numbers(optimize_strings(dataset, datetime_features=["PRODUCT_DATE_FROM", "PRODUCT_DATE_TO"]))

In [16]:
dataset.to_csv("dataset.csv", encoding='latin-1', index=False)